In [63]:
import pdfplumber
import os
import json

from openai import OpenAI




In [64]:
with pdfplumber.open('data/business_license_data.pdf') as pdf:
    reversed_lines = []
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            lines = text.split('\n')
            reversed_lines.extend([line[::-1] for line in lines])  # Reverse each line (RTL support)

all_text_pdf = '\n'.join(reversed_lines)

lines = all_text_pdf.split('\n')
requirements_list = []
for line in lines:
    if any(word in line for word in ["דרישה", "חובה", "על העסק", "נדרש"]):
        requirements_list.append(line.strip())  # Add line if it contains relevant keywords

os.makedirs('output', exist_ok=True)

# Save requirements to JSON
with open('output/requirements.json', 'w', encoding='utf-8') as f:
    json.dump({"requirements": requirements_list}, f, ensure_ascii=False, indent=4)

# Get the API key from the environment
api_key = os.getenv("OPENAI_API_KEY")

# If the API key is set
if api_key:
    # Create the OpenAI client
    client = OpenAI(api_key=api_key)

    # Iterate over the requirements
    for req in requirements_list:
        print(f"🔍 Processing requirement: {req}")

        # Send a request to the OpenAI API
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "אתה מקטלג דרישות רגולטוריות לפי סוג העסק."
                },
                {
                    "role": "user",
                    "content": f"הדרישה היא: \"{req}\".\nתגיד לי איזה תנאים (type וכו') מתאימים לעסק שצריך לקיים את הדרישה הזאת. תשיב במבנה JSON עם 'conditions' ו-'requirement'."
                }
            ]
        )

        # Extract the assistant's response
        answer = response.choices[0].message.content

        # Parse the JSON response
        rule = json.loads(answer)

        # Add the rule to the list
        rules.append(rule)

    # Save rules to JSON
    with open('output/rules.json', 'w', encoding='utf-8') as f:
        json.dump(rules, f, ensure_ascii=False, indent=4)


✅ Created 'output/requirements.json'
🔍 Processing requirement: או בניהולו פועל העסק.


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}